In [143]:
%matplot inline
import pandas as pd
import seaborn as sbn
sbn.set()
import numpy as np
import sqlalchemy
%load_ext sql
engine=sqlalchemy.create_engine("postgresql://nguyli03:@localhost/election")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [144]:
county_results=pd.read_sql("county_results",engine)
cr=county_results

In [145]:
cr.head()

,index,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
0,0,Alabama,AL,Autauga,1001.0,Democrat,Bernie Sanders,544,0.182
1,1,Alabama,AL,Autauga,1001.0,Democrat,Hillary Clinton,2387,0.800
2,2,Alabama,AL,Baldwin,1003.0,Democrat,Bernie Sanders,2694,0.329
3,3,Alabama,AL,Baldwin,1003.0,Democrat,Hillary Clinton,5290,0.647
4,4,Alabama,AL,Barbour,1005.0,Democrat,Bernie Sanders,222,0.078


# Which 10 counties Bernie has the highest vote?

In [146]:
bs=cr[cr.candidate=="Bernie Sanders"].sort_values(by=['votes'],ascending=False).head(10)

In [147]:
dems=cr[(cr.candidate=="Bernie Sanders")|(cr.candidate=="Hillary Clinton")]
dems.head()

,index,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
0,0,Alabama,AL,Autauga,1001.0,Democrat,Bernie Sanders,544,0.182
1,1,Alabama,AL,Autauga,1001.0,Democrat,Hillary Clinton,2387,0.800
2,2,Alabama,AL,Baldwin,1003.0,Democrat,Bernie Sanders,2694,0.329
3,3,Alabama,AL,Baldwin,1003.0,Democrat,Hillary Clinton,5290,0.647
4,4,Alabama,AL,Barbour,1005.0,Democrat,Bernie Sanders,222,0.078


In [148]:
compbh=pd.pivot_table(dems,values='votes',index=['state','county'],columns=['candidate'])

In [149]:
compbh.columns

Index(['Bernie Sanders', 'Hillary Clinton'], dtype='object', name='candidate')

In [150]:
compbh.head()

candidate        Bernie Sanders  Hillary Clinton
state   county                                  
Alabama Autauga             544             2387
        Baldwin            2694             5290
        Barbour             222             2567
        Bibb                246              942
        Blount              395              564

In [151]:
compbh.sort_values("Bernie Sanders", ascending=False).head()

candidate                  Bernie Sanders  Hillary Clinton
state        county                                       
California   Los Angeles           434656           590502
Illinois     Chicago               311225           366954
             Cook Suburbs          212428           249217
Pennsylvania Philadelphia          125688           212785
New York     Brooklyn              116327           174236

# Who won which states?

In [152]:
compbh=pd.pivot_table(dems,values='votes',index='state',columns='candidate', aggfunc=sum)

In [153]:
compbh.head()

candidate,Bernie Sanders,Hillary Clinton
state,,
Alabama,76399,309928
Alaska,440,99
Arizona,163400,235697
Arkansas,64868,144580
California,1502043,1940580


In [154]:
compbh['votediff']=compbh['Bernie Sanders']-compbh['Hillary Clinton']

In [155]:
compbh.sort_values('votediff',ascending=False)

candidate,Bernie Sanders,Hillary Clinton,votediff
state,,,
Wisconsin,567936,432767,135169
Vermont,115863,18335,97528
Oregon,320746,251739,69007
New Hampshire,151584,95252,56332
Utah,61333,15666,45667
West Virginia,123860,86354,37506
Oklahoma,174054,139338,34716
Indiana,335256,303382,31874
Colorado,71928,49256,22672


In [156]:
cf=pd.read_sql('county_facts',engine)

In [157]:
cf.head()

,index,fips,area_name,state_abbreviation,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,...,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
0,0,0,United States,None,318857056,308758105,3.3,308745538,6.2,23.1,...,8.3,28.8,5319456312,4174286516,3917663456,12990,613795732,1046363,3531905.43,87.4
1,1,1000,Alabama,None,4849377,4780127,1.4,4779736,6.1,22.8,...,1.2,28.1,112858843,52252752,57344851,12364,6426342,13369,50645.33,94.4
2,2,1001,Autauga County,AL,55395,54571,1.5,54571,6.0,25.2,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
3,3,1003,Baldwin County,AL,200111,182265,9.8,182265,5.6,22.2,...,1.3,27.3,1410273,0,2966489,17166,436955,1384,1589.78,114.6
4,4,1005,Barbour County,AL,26887,27457,-2.1,27457,5.7,21.2,...,0.0,27.0,0,0,188337,6334,0,8,884.88,31.0


In [158]:
data=pd.merge(cr,cf,left_on='fips',right_on='fips')
data.head()

,index_x,state,state_abbreviation_x,county,fips,party,candidate,votes,fraction_votes,index_y,...,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
0,0,Alabama,AL,Autauga,1001.0,Democrat,Bernie Sanders,544,0.182,2,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
1,1,Alabama,AL,Autauga,1001.0,Democrat,Hillary Clinton,2387,0.800,2,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
2,134,Alabama,AL,Autauga,1001.0,Republican,Ben Carson,1764,0.146,2,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
3,135,Alabama,AL,Autauga,1001.0,Republican,Donald Trump,5387,0.445,2,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
4,136,Alabama,AL,Autauga,1001.0,Republican,John Kasich,421,0.035,2,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8


# How does family size impact the voting decision?

In [159]:
pd.pivot_table(data,values='votes',index="fips",columns='candidate',aggfunc=max).dropna()

candidate,Uncommitted,Ben Carson,Bernie Sanders,Carly Fiorina,Chris Christie,Donald Trump,Hillary Clinton,Jeb Bush,John Kasich,Marco Rubio,Martin O'Malley,Mike Huckabee,Rand Paul,Rick Santorum,Ted Cruz
fips,,,,,,,,,,,,,,,
19001.0,0.0,61.0,132.0,16.0,4.0,104.0,168.0,4.0,2.0,80.0,0.0,11.0,12.0,7.0,104.0
19003.0,0.0,30.0,86.0,16.0,1.0,68.0,113.0,7.0,10.0,47.0,0.0,3.0,8.0,1.0,81.0
19005.0,0.0,77.0,296.0,54.0,4.0,193.0,304.0,11.0,2.0,131.0,0.0,9.0,16.0,6.0,183.0
19007.0,0.0,92.0,226.0,9.0,4.0,292.0,273.0,13.0,4.0,90.0,0.0,44.0,20.0,2.0,269.0
19009.0,0.0,33.0,112.0,4.0,1.0,99.0,187.0,10.0,5.0,56.0,0.0,20.0,11.0,0.0,135.0
19011.0,0.0,171.0,528.0,35.0,23.0,410.0,672.0,11.0,27.0,246.0,0.0,34.0,72.0,9.0,596.0
19013.0,16.0,497.0,3647.0,131.0,104.0,1360.0,3220.0,178.0,53.0,1551.0,16.0,72.0,357.0,20.0,1585.0
19015.0,0.0,177.0,715.0,7.0,40.0,412.0,585.0,31.0,22.0,320.0,0.0,67.0,96.0,19.0,566.0
19017.0,0.0,134.0,544.0,24.0,16.0,347.0,640.0,38.0,50.0,389.0,16.0,19.0,46.0,19.0,408.0


In [171]:
pt=pd.pivot_table(cr,values='votes',index=['state'],columns=['candidate'],aggfunc=max)
pt.iloc[0:,1:].apply(np.max,axis=1)

state
Alabama            67357.0
Alaska               464.0
Arizona           144522.0
Arkansas           34862.0
California        590502.0
Colorado           14624.0
Connecticut         9710.0
Delaware           38580.0
Florida           132527.0
Georgia            82576.0
Hawaii             11458.0
Idaho              20156.0
Illinois          366954.0
Indiana            59649.0
Iowa               12122.0
Kansas             11735.0
Kentucky           64090.0
Louisiana          40601.0
Maine                190.0
Maryland          113912.0
Massachusetts      71422.0
Michigan          163886.0
Mississippi        31386.0
Missouri           89373.0
Montana            17925.0
Nebraska           23754.0
Nevada             20132.0
New Hampshire      38646.0
New Jersey         75071.0
New Mexico         38247.0
New York          177496.0
North Carolina     77927.0
North Dakota          10.0
Ohio              125914.0
Oklahoma           34255.0
Oregon             87247.0
Pennsylvania      2127

In [166]:
pt.iloc[1,2:][pt.iloc[1,2:]==440.0]

Series([], Name: Alaska, dtype: float64)

In [165]:
def findmax(row):
    max_val=np.max(row)
    name=row[row==max_val].index[0]
    return name
pt.iloc[0:,2:].apply(findmax,axis=1)

state
Alabama           Hillary Clinton
Alaska               Donald Trump
Arizona              Donald Trump
Arkansas          Hillary Clinton
California        Hillary Clinton
Colorado           Bernie Sanders
Connecticut       Hillary Clinton
Delaware          Hillary Clinton
Florida           Hillary Clinton
Georgia           Hillary Clinton
Hawaii             Bernie Sanders
Idaho                    Ted Cruz
Illinois          Hillary Clinton
Indiana           Hillary Clinton
Iowa              Hillary Clinton
Kansas                   Ted Cruz
Kentucky          Hillary Clinton
Louisiana         Hillary Clinton
Maine              Bernie Sanders
Maryland          Hillary Clinton
Massachusetts     Hillary Clinton
Michigan          Hillary Clinton
Mississippi       Hillary Clinton
Missouri          Hillary Clinton
Montana              Donald Trump
Nebraska             Donald Trump
Nevada               Donald Trump
New Hampshire      Bernie Sanders
New Jersey        Hillary Clinton
New Mexi

In [164]:
np.max(pt.iloc[1,2:])

464.0